In [ ]:
import os
import cv2
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/NCVPRIPG-writer-verification-main/')
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

# Set the paths to the dataset folders
train_folder = '/content/drive/MyDrive/surya/NCVPRIPG-writer-verification-main/dataset/train'
val_folder = '/content/drive/MyDrive/surya/NCVPRIPG-writer-verification-main/dataset/val'

# Set the path to the validation CSV file
val_csv_file = '/content/drive/MyDrive/surya/NCVPRIPG-writer-verification-main/dataset/val.csv'

# Define the image size for resizing
image_size = (64, 64)

# Preprocess the images
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

# Load the training data
train_images = []
train_labels = []

for folder_name in os.listdir(train_folder):
    folder_path = os.path.join(train_folder, folder_name)
    if os.path.isdir(folder_path):
        image_paths = sorted(os.listdir(folder_path))
        for i in range(len(image_paths) - 1):
            image1_path = os.path.join(folder_path, image_paths[i])
            image2_path = os.path.join(folder_path, image_paths[i + 1])
            image1 = preprocess_image(image1_path)
            image2 = preprocess_image(image2_path)
            train_images.append(image1)
            train_images.append(image2)
            train_labels.append(1)  # Same writer label
            train_labels.append(1)  # Same writer label

# Load the validation data
val_data = pd.read_csv(val_csv_file)
val_image_pairs = []

for i, row in val_data.iterrows():
    image1_path = os.path.join(val_folder, row['img1_name'])
    image2_path = os.path.join(val_folder, row['img2_name'])
    image1 = preprocess_image(image1_path)
    image2 = preprocess_image(image2_path)
    val_image_pairs.append((image1, image2))

# Convert the training data to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Split the training data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42)

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))

Mounted at /content/drive


In [ ]:
# Make predictions on the validation set
val_predictions = []

for pair in val_image_pairs:
    image1, image2 = pair
    prediction = model.predict(np.array([image1, image2]))
    val_predictions.append(prediction[0][0])

# Calculate F1 score and AUC
val_true_labels = val_data['label']
f1 = f1_score(val_true_labels, np.round(val_predictions))
auc = roc_auc_score(val_true_labels, val_predictions)

# Create the submission CSV file
submission_file = '/content/drive/MyDrive/surya/NCVPRIPG-writer-verification-main/dataset/submission.csv'
submission_data = val_data.copy()
submission_data['proba'] = val_predictions
submission_data.to_csv(submission_file, index=False)

# Print F1 score and AUC
print("F1 score:", f1)
print("AUC:", auc)